# Data Cleaning
Let's go ahead and jump right into the data and see what we find here fresh off the scraping press!

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
# read in our data
odd_df = pd.read_csv('../data/oddlysatisfying.csv')
mild_df = pd.read_csv('../data/mildlyinfuriating.csv')
print ('r/oddlysatisfying shape: ',odd_df.shape)
print ('r/mildlyinfuriating shape' ,mild_df.shape)

r/oddlysatisfying shape:  (99971, 4)
r/mildlyinfuriating shape (99983, 4)


## Check for sheNaNigans
First off lets check for null values that may have come in during data collection.

In [3]:
# check nulls from oddlysatisfying data
odd_df.isna().sum()

author         0
body           1
created_utc    0
subreddit      0
dtype: int64

In [4]:
# looks like we have a single null value. Let's take a look at it and see if there's anything particularly interesting about it
odd_df[odd_df['body'].isna() == True]

,author,body,created_utc,subreddit
37990,MrKotlet,NaN,1634504344,oddlysatisfying


In [5]:
# check nulls from mildlyinfuriating data
mild_df.isna().sum()

author         0
body           0
created_utc    0
subreddit      0
dtype: int64

Looks like there wasn't much missing in our data at first glance. We'll likely create more nulls when we clean up our comments so we won't do anything with our single null value just yet.

## First looks at common comments
Next we'll check for our most frequent matching comments are and see what they look like in our data.

In [6]:
#check for repeat comments in oddlysatisfying
odd_df['body'].value_counts()

[deleted]                                                                                                                                                                                                                        2774
[removed]                                                                                                                                                                                                                        2075
u/savevideo                                                                                                                                                                                                                       570
u/savevideobot                                                                                                                                                                                                                    125
This is great                                                                   

In [7]:
#check for repeat comments in mildlyinfuriating
mild_df['body'].value_counts()[:2]

[removed]    2532
[deleted]    1990
Name: body, dtype: int64

Deleted and removed comments don't seem like they are going to be great indicators since they are similarly present between both datasets. Lets go ahead and convert them to nulls and drop them for now. We also have some \n characters that need to be dealt with and some links (embedded and otherwise) that don't help us much when trying to determine where the comments have come from.

In [8]:
# examples from oddlysatisfying
odd_df['contains link'] = [1 if 'http' in str(value) else 0 for value in odd_df['body']]
odd_df[odd_df['contains link'] == 1].head()

,author,body,created_utc,subreddit,contains link
141,circ_le_jerk_69,https://www.reddit.com/r/todayilearned/comment...,1635374639,oddlysatisfying,1
152,BusyBusyBizzy,https://www.youtube.com/watch?v=IVEISV-AaJc\r ...,1635374403,oddlysatisfying,1
320,Lucifang,[Mr Squiggle](https://m.youtube.com/watch?v=Hw...,1635369834,oddlysatisfying,1
381,BusyBusyBizzy,from u/artmanee\r \n\r \nhttps://vm.tiktok.c...,1635368401,oddlysatisfying,1
391,Aries2203,Usually means someone who is overly fussy or p...,1635368004,oddlysatisfying,1


In [9]:
# examples from mildlyinfuriating
mild_df['contains link'] = [1 if 'http' in value else 0 for value in mild_df['body']]
mild_df[mild_df['contains link'] == 1].head()

,author,body,created_utc,subreddit,contains link
114,ducktheRedditapp,Or get a soda stealing suit\n\nhttps://youtu.b...,1635374745,mildlyinfuriating,1
130,darnbot,What a ***darn*** shame...\n\n---\n^^DarnCount...,1635374691,mildlyinfuriating,1
131,RamboGoesMeow,[I prefer child size cups.](https://youtu.be/I...,1635374690,mildlyinfuriating,1
232,Midcenturybirdhouse,"That's rookie numbers, friend. If you're going...",1635374194,mildlyinfuriating,1
267,strikeout44,"So I did a little research. It’s illegal, acco...",1635374030,mildlyinfuriating,1


In [10]:
#removing columns that were added for example purposes
odd_df.drop(columns='contains link',inplace=True)
mild_df.drop(columns='contains link',inplace=True)

Since comments may contain multiple links I designed a recursive function which detects if a link is present and then sends the second half of the string back into the function to check for more links.

In [11]:
#recursive function to find all instances of embeded links in a comment
def embeded_link_cleaner(text):
    if '](' in text:
        # separates the text based on the first indicator and sends the second half back in to find additional links
        text = text[:text.index('](')+1] + embeded_link_cleaner(text[text.index('](')+1:]) 
        
        #find all text before and after the (https://...)
        pre_link = text[:text.index('](')+1]
        post_link = text[text[text.index('](')+1:].index(')')+len(pre_link)+1:] # text[text.index('](')+1:].index(')') returns the length of the (https://...) portion
        
        # return the text without the embedded link
        return (pre_link + post_link).replace('[','').replace(']','')
    else:
        # if there are no embeded links just return the input text
        return text

In [12]:
# removes all link and \n instances from every value in a pandas series
def comment_cleaner(series):
    
    clean_series = []
        
    for value in series:
        # clean out all embedded links and replace new line args with spaces
        value = embeded_link_cleaner(str(value)).replace('\n',' ')
        
        # set values that contain no actual language information to nulls
        if value in ['[deleted]','[removed]','[Update]']:
            value = np.NaN
        
        # remove non embedded links
        if 'https' in str(value):
            value_list = value.split(' ')
            
            if len(value_list) <= 1:
                value = np.NaN
            else:
                value = ' '.join([word for word in value_list if 'https' not in word])          
        
        clean_series.append(value)
        
    return clean_series

Since we've made our cleaning instruments lets scrub up our data!

In [13]:
# send both dataframes in for cleaning
odd_df['body'] = comment_cleaner(odd_df['body'])
mild_df['body'] = comment_cleaner(mild_df['body'])

# drop any nulls that resulted from the collection and cleaning process
odd_df.dropna(inplace=True)
mild_df.dropna(inplace=True)

# check the shape of our cleaned data sets and verify nulls have been cleaned out
print (odd_df.shape,odd_df.isna().sum().sum())
print (mild_df.shape, mild_df.isna().sum().sum())

(94899, 4) 0
(95272, 4) 0


It looks like we did lose a small chunk from our original collection. We'll likely have to trim down even further due to computational constraints later so it won't be an issue here!

Let's save our work so we can start looking for patterns in our data!

In [14]:
# export dataframes into .csv files for use in other notebooks
odd_df.to_csv('../data/satisfyingly_clean.csv', index=False)
mild_df.to_csv('../data/infuriatingly_clean.csv',index=False)